#### Import necessary packages

In [1]:
import matsim
import os 
import pandas as pd
#import plotly.express as px 
# AttributeError: module 'numpy' has no attribute 'bool8'
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from collections import defaultdict
import plotly
import kaleido
%matplotlib inline
import plotly.io as pio
import numpy as np

#### Number of stuck time violations at iteration 0

In [3]:
# Number of stuck vehicles
def countStuckVehicles(pathToEvents):
    events_file = pathToEvents
    #"/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.output_events.xml.gz"

    #print("reading events:", events_file)

    # Read events - filter and return the listed event types only
    events = matsim.event_reader(
        events_file,
        types="stuckAndContinue",
    )

    stuck_person = defaultdict(list)

    # Loop on all filtered events
    for event in events:
        if event["type"] == "stuckAndContinue":
            stuck_person[event["person"]].append(event["legMode"])
    
    count = 0 
    for key in stuck_person.keys():
        count+= len(stuck_person[key])
    
    return count


##### 1 pct

In [6]:
flowCapF = ["0.01"]
storCapF =  ["0.01", "0.03162"]
stuckTimes = ["30.0", "3000.0"]

stuck_time_violations_1pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.01") & (sCf == "0.01") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(sampleNr) + "-fCf_sCf_0.01_gS_default_3765.0.events.xml.gz"
                    alpha = 1.0

                elif((fCf == "0.01") & (sCf ==  "0.03162") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(sampleNr) + "-fCf_0.01_sCf_0.03162_gS_default_3765.0.events.xml.gz"
                    alpha = 0.75
                
                elif((fCf == "0.01") & (sCf == "0.01") & (sT == "3000.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf  + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-1-pct-" + str(sampleNr) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.0.events.xml.gz"
                    alpha = 1.0

                elif((fCf == "0.01") & (sCf ==  "0.03162") & (sT == "3000.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-1-pct-" + str(sampleNr) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.0.events.xml.gz"
                    alpha = 0.75
                else: 
                    print("case not found")
                    break
                # count the number 
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "1-pct", 'sample_nr': sampleNr, 'alpha' : alpha, 'stuck_time': float(sT), 'global_seed': "default"}, index = [counter])
                stuck_time_violations_1pct_it0 = pd.concat([stuck_time_violations_1pct_it0, temp], axis = 0)
                counter += 1
    


In [8]:
#  1 pct random seed
stuck_veh_1pct_rGs_it0 = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        # this case has alreade been counted
        global_seed = "rnd_" + str(seed)
        # insert number of stuck time violations from the first 1 pct sample
        temp = {'n_stuck_veh': stuck_time_violations_1pct_it0["n_stuck_veh"].iloc[0], 'sample_size': '1-pct', 'sample_nr': 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        stuck_veh_1pct_rGs_it0 = pd.concat([stuck_veh_1pct_rGs_it0, temp])
    
    elif (seed == 3254):
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/ITERS/it.0/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.0.events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp1 = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': '1-pct', 'sample_nr': 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(3254)])
        stuck_veh_1pct_rGs_it0 = pd.concat([stuck_veh_1pct_rGs_it0, temp1], axis = 0)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765/ITERS/it.0/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.0.events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp2 = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': '1-pct', 'sample_nr': 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)] )
        stuck_veh_1pct_rGs_it0 = pd.concat([stuck_veh_1pct_rGs_it0, temp2], axis = 0)

In [ ]:
stuck_time_violations_1pct_it0 = pd.concat([stuck_time_violations_1pct_it0, stuck_veh_1pct_rGs_it0], axis = 0, ignore_index= True)
stuck_time_violations_1pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_pct_it0.csv', index = False) 

##### 5 pct

In [ ]:
flowCapF = ["0.05"]
storCapF =  ["0.05", "0.10574"]
stuckTimes = ["30.0", "600.0"]

stuck_time_violations_5pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.05") & (sCf == "0.05") & (sT == "30.0")):
                    if (sampleNr == 6):
                        path  = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.0.events.xml.gz"
                    else: 
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765.0.events.xml.gz"
                    alpha = 1.0
                    
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "30.0")):
                    if(sampleNr == 6):
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.0.events.xml.gz"
                    else:
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_3765.0.events.xml.gz"
                    alpha = 0.75
                elif((fCf == "0.05") & (sCf == "0.05") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.0.events.xml.gz"
                    alpha = 1.0
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.0.events.xml.gz"
                    alpha = 0.75
                
                else: 
                    print("case not found")
                    break
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "5-pct", 'sample_nr': sampleNr, 'alpha' : alpha, 'stuck_time': float(sT), 'global_seed': "default"}, index = [counter])
                stuck_time_violations_5pct_it0 = pd.concat([stuck_time_violations_5pct_it0, temp], axis = 0)
                counter += 1
                

In [ ]:
# 5 pct random global seed
stuck_time_violations_5pct_it0_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'n_stuck_veh': stuck_time_violations_5pct_it0["n_stuck_veh"].iloc[0], 'sample_size': '5-pct', "sample_nr": 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        stuck_time_violations_5pct_it0_rGs = pd.concat([stuck_time_violations_5pct_it0_rGs, temp])
    else:
        global_seed = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/ITERS/it.0/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765.0.events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "5-pct", 'sample_nr': sampleNr, 'alpha' : 1.0, 'stuck_time': 30.0, 'global_seed': global_seed}, index = [counter])
        stuck_time_violations_5pct_it0_rGs = pd.concat([stuck_time_violations_5pct_it0_rGs, temp], axis = 0)
        counter += 1


In [ ]:
# concat and write output file
stuck_time_violations_5pct_it0 = pd.concat([stuck_time_violations_5pct_it0, stuck_time_violations_5pct_it0_rGs], axis = 0, ignore_index= True)
stuck_time_violations_5pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_5_pct_it0.csv', index = False)

##### 10 pct

In [4]:
flowCapF = ["0.1"]
storCapF =  ["0.1", "0.17783"]
stuckTimes = ["30.0", "300.0"]

stuck_time_violations_10pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.1") & (sCf == "0.1") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765.0.events.xml.gz"
                    alpha = 1.0       
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_3765.0.events.xml.gz"
                    alpha = 0.75
                elif((fCf == "0.1") & (sCf == "0.1") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.0.events.xml.gz"
                    alpha = 1.0
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.0.events.xml.gz"
                    alpha = 0.75  
                else: 
                    print("case not found")
                    break
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "10-pct", 'sample_nr': sampleNr, 'alpha' : alpha, 'stuck_time': float(sT), 'global_seed': "default"}, index = [counter])
                stuck_time_violations_10pct_it0 = pd.concat([stuck_time_violations_10pct_it0, temp], axis = 0)
                counter += 1


In [6]:
# write output file
stuck_time_violations_10pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_10_pct_it0.csv', index = False) 

##### 25 pct

In [21]:
flowCapF = ["0.25"]
storCapF =  ["0.25", "0.35355"]
stuckTimes = ["30.0", "120.0"]

stuck_time_violations_25pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.25") & (sCf == "0.25") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.0.events.xml.gz"
                    alpha = 1.0      
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.0.events.xml.gz"
                    alpha  = 0.75
                elif((fCf == "0.25") & (sCf == "0.25") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-" + str(sampleNr) + "-fCf_sCF_" + fCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.0.events.xml.gz"
                    alpha = 1.0
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-" + str(sampleNr) + "-fCf_" + fCf + "_sCF_" + sCf + "_gS_4711_sT_" + sT + "_3765/ITERS/it.0/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.0.events.xml.gz" 
                    alpha = 0.75
                else: 
                    print("case not found")
                    break
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "25-pct", 'sample_nr': sampleNr, 'alpha' : alpha, 'stuck_time': float(sT), 'global_seed': "default"}, index = [counter])
                stuck_time_violations_25pct_it0 = pd.concat([stuck_time_violations_25pct_it0, temp], axis = 0)
                counter += 1

In [22]:
stuck_time_violations_25pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_25_pct_it0.csv', index = False)

##### 50 pct 

In [25]:
flowCapF = ["0.5"]
storCapF =  ["0.5", "0.5946"]
stuckTimes = ["30.0", "60.0"]

stuck_time_violations_50pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.5") & (sCf == "0.5") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.0.events.xml.gz"
                    alpha = 1.0       
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.0.events.xml.gz"
                    alpha = 0.75
                elif((fCf == "0.5") & (sCf == "0.5") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.0.events.xml.gz"
                    alpha = 1.0
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.0.events.xml.gz"
                    alpha = 0.75 
                else: 
                    print("case not found")
                    break
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "50-pct", 'sample_nr': sampleNr, 'alpha' : alpha, 'stuck_time': float(sT), 'global_seed': "default"}, index = [counter])
                stuck_time_violations_50pct_it0 = pd.concat([stuck_time_violations_50pct_it0, temp], axis = 0)
                counter += 1
                

In [26]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711__3765.0.events.xml.gz"
stuck_veh = countStuckVehicles(path)
temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "25-pct-doubled", 'sample_nr': 1, 'alpha' : 1.0, 'stuck_time': 30.0, 'global_seed': "default"}, index = [counter])
stuck_time_violations_50pct_it0 = pd.concat([stuck_time_violations_50pct_it0, temp], axis = 0)

In [27]:
stuck_time_violations_50pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_50_pct_it0.csv', index = False)

In [31]:
stuck_time_violations_50pct_it0

,n_stuck_veh,sample_size,sample_nr,alpha,stuck_time,global_seed
0,15224,50-pct,1,1.00,30.0,default
1,15137,50-pct,1,1.00,60.0,default
2,15126,50-pct,1,0.75,30.0,default
3,15109,50-pct,1,0.75,60.0,default
4,16258,25-pct-doubled,1,1.00,30.0,default


##### 100 pct

In [ ]:
flowCapF = ["1.0"]
storCapF =  ["1.0"]
stuckTimes = ["30.0"]

stuck_time_violations_100pct_it0 = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.0.events.xml.gz"
                stuck_veh = countStuckVehicles(path)
                temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "100-pct", 'sample_nr': sampleNr, 'alpha' : 1.0, 'stuck_time': 30.0, 'global_seed': "default"}, index = [counter])
                stuck_time_violations_100pct_it0 = pd.concat([stuck_time_violations_100pct_it0, temp], axis = 0)

In [ ]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25.0-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-25-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711__3765.0.events.xml.gz"
stuck_veh = countStuckVehicles(path)
temp = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': "25-pct-quadrupled", 'sample_nr': 1, 'alpha' : 1.0, 'stuck_time': 30.0, 'global_seed': "default"}, index = [counter])
stuck_time_violations_100pct_it0 = pd.concat([stuck_time_violations_100pct_it0, temp], axis = 0)

In [ ]:
stuck_time_violations_100pct_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_100pct_it0.csv', index = False)

##### Concat and write output file

In [ ]:
stuck_time_violations_1pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_pct_it0.csv")
stuck_time_violations_5pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_5_pct_it0.csv")
stuck_time_violations_10pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_10_pct_it0.csv")
stuck_time_violations_25pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_25_pct_it0.csv")
stuck_time_violations_50pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_50_pct_it0.csv")
stuck_time_violations_100pct_it0 = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_100_pct_it0.csv")


In [ ]:
# concat all
stuck_time_violations_1_100_it_0= pd.concat([stuck_time_violations_1pct_it0, stuck_time_violations_5pct_it0, stuck_time_violations_10pct_it0, stuck_time_violations_25pct_it0, stuck_time_violations_50pct_it0, stuck_time_violations_100pct_it0 ], axis = 0)
# write all to csv
stuck_time_violations_1_100_it_0.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_100_it_0.csv", index = False)


#### Number of Link leave events

#### Executed Scores

##### Original Plans, before Simulation

##### 1 pct

In [4]:
#### scores ####
# 1pct, alpha = 1
scores_1pct_beforeSim = pd.DataFrame()
for sampleNr in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(sampleNr) +"-fCf_sCF_0.01_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(sampleNr) + "-fCf_sCf_0.01_gS_default_3765.0.plans.xml.gz"
    plans = matsim.plan_reader(path, selected_plans_only = True)
    experienced_scores = []
    id = []
    for person, plan in plans:
        experienced_scores.append(plan.attrib['score'])
        id.append(person.attrib['id'])
    experienced_scores = pd.DataFrame({'id': id, 'exp_score': pd.to_numeric(experienced_scores)})
    average = np.mean(experienced_scores['exp_score'])
    df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [sampleNr])
    scores_1pct_beforeSim = pd.concat([scores_1pct_beforeSim, df], axis = 0)

##### 5 pct

#### Iteration 0

##### 1 pct

In [4]:
#### scores ####
# 1pct, alpha = 1
scores_1pct = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_sCF_0.01_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(elem) + "-fCf_sCf_0.01_gS_default_3765.0.experienced_plans.xml.gz"
    plans = matsim.plan_reader(path, selected_plans_only = True)
    experienced_scores = []
    id = []
    for person, plan in plans:
        experienced_scores.append(plan.attrib['score'])
        id.append(person.attrib['id'])
    experienced_scores = pd.DataFrame({'id': id, 'exp_score': pd.to_numeric(experienced_scores)})
    average = np.mean(experienced_scores['exp_score'])
    df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "1-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_1pct = pd.concat([scores_1pct, df], axis = 0)

In [5]:
# 1 pct, alpha = 0.75
scores_1pct_sCf = pd.DataFrame()
for elem in range(1,11,1):
    # /home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_0.01_sCF_0.03162_gS_default_3765/ITERS/it.0/lausitz-1pct-1-fCf_0.01_sCf_0.03162_gS_default_3765.0.experienced_plans.xml.gz
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" +str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(elem) + "-fCf_0.01_sCf_0.03162_gS_default_3765.0.experienced_plans.xml.gz"
    plans = matsim.plan_reader(path, selected_plans_only = True)
    experienced_scores = []
    id = []
    for person, plan in plans:
        experienced_scores.append(plan.attrib['score'])
        id.append(person.attrib['id'])
    experienced_scores = pd.DataFrame({'id': id, 'exp_score': pd.to_numeric(experienced_scores)})
    average = np.mean(experienced_scores['exp_score'])
    df = pd.DataFrame({"avg_executed_it_0":average, "sample_size": "1-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sCf = pd.concat([scores_1pct_sCf, df], axis = 0)

In [6]:
# 1 pct, alpha 1, random global seed
#  1 pct random seed
scores_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # insert number of stuck time violations from the first 1 pct sample
        df = pd.DataFrame({"avg_executed_it_0":scores_1pct["avg_executed_it_0"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df], axis = 0)
    
    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/ITERS/it.0/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.0.experienced_plans.xml.gz"
        plans = matsim.plan_reader(path, selected_plans_only = True)
        experienced_scores = []
        for person, plan in plans:
            experienced_scores.append(plan.attrib['score'])

        experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
        average = np.mean(experienced_scores['exp_score'])
        df = pd.DataFrame({"avg_executed_it_0":average, "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df], axis = 0)
    else:
        global_seed = "rnd_" + str(seed)
        # /home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_6003_3765/ITERS/it.0/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_6003_3765.0.experienced_plans.xml.gz
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765/ITERS/it.0/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.0.experienced_plans.xml.gz"
        plans = matsim.plan_reader(path, selected_plans_only = True)
        experienced_scores = []
        for person, plan in plans:
            experienced_scores.append(plan.attrib['score'])
        
        experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
        average = np.mean(experienced_scores['exp_score'])
        df = pd.DataFrame({"avg_executed_it_0":average, "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df], axis = 0)

In [7]:
# 1 pct, alpha = 1, sT scaled
scores_1pct_sT =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/ITERS/it.0/lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.0.experienced_plans.xml.gz"
    plans = matsim.plan_reader(path, selected_plans_only = True)
    experienced_scores = []
    for person, plan in plans:
        experienced_scores.append(plan.attrib['score'])
    experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
    average = np.mean(experienced_scores['exp_score'])
    df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "1-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 3000.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sT = pd.concat([scores_1pct_sT, df], axis = 0)


In [8]:
# 1 pct, alpha = 0.75, sT scaled
scores_1pct_sT_sCf =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/ITERS/it.0/lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.0.experienced_plans.xml.gz"
    plans = matsim.plan_reader(path, selected_plans_only = True)
    experienced_scores = []
    for person, plan in plans:
        experienced_scores.append(plan.attrib['score'])
    experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
    average = np.mean(experienced_scores['exp_score'])
    df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "1-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 3000.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sT_sCf = pd.concat([scores_1pct_sT_sCf, df], axis = 0)

In [9]:
scores_1pct_all_it0 = pd.concat([scores_1pct, scores_1pct_sCf, scores_1pct_rGs, scores_1pct_sT, scores_1pct_sT_sCf])

In [10]:
scores_1pct_all_it0.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_1pct_samples_it0.csv', index = False) 

##### 5 pct

In [11]:
flowCapF = ["0.05"]
storCapF =  ["0.05", "0.10574"]
stuckTimes = ["30.0", "600.0"]

scores_5pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.05") & (sCf == "0.05") & (sT == "30.0")):
                    if (sampleNr == 6):
                        path  = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.0.experienced_plans.xml.gz"
                    else: 
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765.0.experienced_plans.xml.gz"
                    
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "30.0")):
                    if(sampleNr == 6):
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.0.experienced_plans.xml.gz"
                    else:
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.05") & (sCf == "0.05") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-"+ str(sampleNr) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.0.experienced_plans.xml.gz"
                
                else: 
                    print("case not found")
                    break
                # read experienced plans file
                plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
                experienced_scores = []
                # store all executed scores from plans file in list
                for person, plan in plans:
                    experienced_scores.append(plan.attrib['score'])
                experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
                # calculate average of these scores
                average = np.mean(experienced_scores['exp_score'])
                if (sCf == "0.10574"): 
                    alpha = 0.75
                elif( sCf == "0.05"):
                    alpha = 1.0
                # create df for specific case and store it in scores_5pct
                df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "5-pct", "sample_nr": sampleNr, "alpha": alpha, "stuck_time": float(sT), "global_seed": "default" }, index = [counter])
                scores_5pct = pd.concat([scores_5pct, df], axis = 0)
                counter += 1


In [12]:
# 5 pct random global seed
scores_5pct_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'avg_executed_it_0': scores_5pct["avg_executed_it_0"].iloc[0], 'sample_size': '5-pct', "sample_nr": 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        scores_5pct_rGs = pd.concat([scores_5pct_rGs, temp])
    else:
        global_seed = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_"+ str(seed) + "_3765/ITERS/it.0/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765.0.experienced_plans.xml.gz"
        plans = matsim.plan_reader(path, selected_plans_only = True)
        experienced_scores = []
        for person, plan in plans:
            experienced_scores.append(plan.attrib['score'])
        experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
        average = np.mean(experienced_scores['exp_score'])
        temp = {'avg_executed_it_0': average, 'sample_size': '5-pct', "sample_nr": 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        df = pd.DataFrame(data=temp, index=[rGs.index(seed)])
        scores_5pct_rGs = pd.concat([scores_5pct_rGs, df], axis = 0)


In [13]:
# concat dfs
scores_5pct_all_it0 = pd.concat([scores_5pct, scores_5pct_rGs])

In [14]:
# write csv
scores_5pct_all_it0.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_5pct_samples_it0.csv", index = False)

##### 10 pct

In [15]:
flowCapF = ["0.1"]
storCapF =  ["0.1", "0.17783"]
stuckTimes = ["30.0", "300.0"]

scores_10pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.1") & (sCf == "0.1") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765.0.experienced_plans.xml.gz"       
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.1") & (sCf == "0.1") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.0.experienced_plans.xml.gz"  
                else: 
                    print("case not found")
                    break
                # read experienced plans file
                plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
                experienced_scores = []
                # store all executed scores from plans file in list
                for person, plan in plans:
                    experienced_scores.append(plan.attrib['score'])
                experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
                # calculate average of these scores
                average = np.mean(experienced_scores['exp_score'])
                if (sCf == "0.17783"): 
                    alpha = 0.75
                elif( sCf == "0.1"):
                    alpha = 1.0
                # create df for specific case and store it in scores_10pct
                df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "10-pct", "sample_nr": sampleNr, "alpha": alpha, "stuck_time": float(sT), "global_seed": "default" }, index = [counter])
                scores_10pct = pd.concat([scores_10pct, df], axis = 0)
                counter += 1


In [16]:
# write csv
scores_10pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_10pct_samples_it0.csv", index = False)

##### 25 pct

In [17]:
flowCapF = ["0.25"]
storCapF =  ["0.25", "0.35355"]
stuckTimes = ["30.0", "120.0"]

scores_25pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.25") & (sCf == "0.25") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.0.experienced_plans.xml.gz"       
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.25") & (sCf == "0.25") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/ITERS/it.0/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/ITERS/it.0/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.0.experienced_plans.xml.gz"  
                else: 
                    print("case not found")
                    break
                # read experienced plans file
                plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
                experienced_scores = []
                # store all executed scores from plans file in list
                for person, plan in plans:
                    experienced_scores.append(plan.attrib['score'])
                experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
                # calculate average of these scores
                average = np.mean(experienced_scores['exp_score'])
                if (sCf == "0.35355"): 
                    alpha = 0.75
                elif(sCf == "0.25"):
                    alpha = 1.0
                # create df for specific case and store it in scores_25pct
                df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "25-pct", "sample_nr": sampleNr, "alpha": alpha, "stuck_time": float(sT), "global_seed": "default" }, index = [counter])
                scores_25pct = pd.concat([scores_25pct, df], axis = 0)
                counter += 1


In [18]:
# write csv
scores_25pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_25pct_samples_it0.csv", index = False)

##### 50 pct

In [19]:
flowCapF = ["0.5"]
storCapF =  ["0.5", "0.5946"]
stuckTimes = ["30.0", "60.0"]

scores_50pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.5") & (sCf == "0.5") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.0.experienced_plans.xml.gz"       
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.5") & (sCf == "0.5") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.0.experienced_plans.xml.gz"
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.0.experienced_plans.xml.gz"  
                else: 
                    print("case not found")
                    break
                # read experienced plans file
                plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
                experienced_scores = []
                # store all executed scores from plans file in list
                for person, plan in plans:
                    experienced_scores.append(plan.attrib['score'])
                experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
                # calculate average of these scores
                average = np.mean(experienced_scores['exp_score'])
                if (sCf == "0.5946"): 
                    alpha = 0.75
                elif( sCf == "0.1"):
                    alpha = 1.0
                # create df for specific case and store it in scores_10pct
                df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "50-pct", "sample_nr": sampleNr, "alpha": alpha, "stuck_time": float(sT), "global_seed": "default" }, index = [counter])
                scores_50pct = pd.concat([scores_50pct, df], axis = 0)
                counter += 1


In [20]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711__3765.0.experienced_plans.xml.gz"
plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
experienced_scores = []
                # store all executed scores from plans file in list
for person, plan in plans:
    experienced_scores.append(plan.attrib['score'])
experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
# calculate average of these scores
average = np.mean(experienced_scores['exp_score'])
# create df for specific case and store it in scores_10pct
df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "25-pct-doubled", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [counter])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)

In [21]:
# write csv 
scores_50pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_50pct_samples_it0.csv", index = False)

##### 100 pct 

In [22]:
flowCapF = ["1.0"]
storCapF =  ["1.0"]
stuckTimes = ["30.0"]

scores_100pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.0.experienced_plans.xml.gz"
                # read experienced plans file
                plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
                experienced_scores = []
                # store all executed scores from plans file in list
                for person, plan in plans:
                    experienced_scores.append(plan.attrib['score'])
                experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
                # calculate average of these scores
                average = np.mean(experienced_scores['exp_score'])
                alpha = 1.0
                # create df for specific case and store it in scores_10pct
                df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "100-pct", "sample_nr": sampleNr, "alpha": alpha, "stuck_time": float(sT), "global_seed": "default" }, index = [counter])
                scores_100pct = pd.concat([scores_100pct, df], axis = 0)
                


In [23]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25.0-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-25-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711__3765.0.experienced_plans.xml.gz"
plans = matsim.plan_reader(path, selected_plans_only = True)
                # create list to store scores in
experienced_scores = []
                # store all executed scores from plans file in list
for person, plan in plans:
    experienced_scores.append(plan.attrib['score'])
experienced_scores = pd.DataFrame({'exp_score': pd.to_numeric(experienced_scores)})
# calculate average of these scores
average = np.mean(experienced_scores['exp_score'])
# create df for specific case and store it in scores_10pct
df = pd.DataFrame({"avg_executed_it_0": average, "sample_size": "25-pct-quadrupled", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [counter])
scores_100pct = pd.concat([scores_100pct, df], axis = 0)

##### Concat and write output files

In [24]:
# concat all
scores_1_100 = pd.concat([scores_1pct_all_it0, scores_5pct_all_it0, scores_10pct, scores_25pct, scores_50pct, scores_100pct ], axis = 0)
# write all to csv
scores_1_100.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/exp_scores_all_1_100pct_samples_it0.csv", index = False)


#### Average Traveltime and distance

##### 1 pct

In [34]:
#### average travel time and average traveled distance ####
# trav_time and traveled_distance
# 1 pct, alpha = 1
flowCapF = ["0.01"]
storCapF = ["0.01", "0.03162"]
stuckTimes = ["30.0", "3000.0"]

avg_trav_time_1pct = pd.DataFrame()
avg_trav_dist_1pct = pd.DataFrame()
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                if((fCf == "0.01") & (sCf == "0.01") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(sampleNr) + "-fCf_sCF_" + fCf + "_gS_default_3765/ITERS/it.0/lausitz-1pct-" + str(sampleNr) + "-fCf_sCf_" + fCf + "_gS_default_3765.0.trips.csv.gz"   
                elif((fCf == "0.01") & (sCf ==  "0.03162") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1pct-"+ str(sampleNr) +  "-fCf_0.01_sCF_" + sCf + "_gS_default_3765/ITERS/it.0/lausitz-1pct-" +str(sampleNr) + "-fCf_0.01_sCf_" + sCf + "_gS_default_3765.0.trips.csv.gz"
                elif((fCf == "0.01") & (sCf == "0.01") & (sT == "3000.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(sampleNr) +"-fCf_sCF_" + fCf + "_gS_4711_sT_3000.0_3765/ITERS/it.0/lausitz-1-pct-" + str(sampleNr) + "-fCf_sCF_"+ fCf + "_gS_4711_sT_3000.0_3765.0.trips.csv.gz"
                elif((fCf == "0.01") & (sCf ==  "0.03162") & (sT == "3000.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(sampleNr) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/ITERS/it.0/lausitz-1-pct-" + str(sampleNr) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.0.trips.csv.gz"  
                else: 
                    print("case not found")
                    break
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                if(sCf == "0.03162"):
                    alpha = 0.75
                else: 
                    alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_1pct = pd.concat([avg_trav_time_1pct, df_avg_trav_time], axis = 0)

                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_1pct = pd.concat([avg_trav_dist_1pct, df_avg_trav_dist], axis = 0)

In [35]:
# 1 pct rGs
avg_trav_time_1pct_rGs = pd.DataFrame()
avg_trav_dist_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # Create Data frame and insert first value from avg_trav_time_1pct
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time_1pct["avg_trav_time"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1])
        
        # Create Data frame and insert first value from avg_trav_dist_1pct
        df1 = pd.DataFrame({"avg_trav_dist":avg_trav_dist_1pct["avg_trav_dist"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df1])

    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/ITERS/it.0/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.0.trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = np.mean(temp["traveled_distance"])
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df_avg_trav_dist], axis = 0)

    else:
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765/ITERS/it.0/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.0.trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = np.mean(temp["traveled_distance"])
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df_avg_trav_dist], axis = 0)

In [36]:
avg_trav_time_1pct_all = pd.concat([avg_trav_time_1pct, avg_trav_time_1pct_rGs], axis = 0 , ignore_index= True)
avg_trav_time_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_1pct_samples_it0.csv', index = False) 
avg_trav_dist_1pct_all = pd.concat([avg_trav_dist_1pct, avg_trav_dist_1pct_rGs ], axis = 0 , ignore_index= True)
avg_trav_dist_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_1pct_samples_it0.csv', index = False) 

##### 5pct

In [37]:
flowCapF = ["0.05"]
storCapF =  ["0.05", "0.10574"]
stuckTimes = ["30.0", "600.0"]

avg_trav_time_5pct = pd.DataFrame()
avg_trav_dist_5pct = pd.DataFrame()

for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.05") & (sCf == "0.05") & (sT == "30.0")):
                    if (sampleNr == 6):
                        path  = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.0.trips.csv.gz"
                    else: 
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765.0.trips.csv.gz"
                    
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "30.0")):
                    if(sampleNr == 6):
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.0.trips.csv.gz"
                    else:
                        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_3765.0.trips.csv.gz"
                elif((fCf == "0.05") & (sCf == "0.05") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.0.trips.csv.gz"
                elif((fCf == "0.05") & (sCf ==  "0.10574") & (sT == "600.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/ITERS/it.0/lausitz-5-pct-" + str(sampleNr) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.0.trips.csv.gz"
                
                else: 
                    print("case not found")
                    break
                # calculate average travel time
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                if(sCf == "0.10574"):
                    alpha = 0.75
                else: 
                    alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_5pct = pd.concat([avg_trav_time_5pct, df_avg_trav_time], axis = 0)
                # calculate average traveled distance
                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_5pct = pd.concat([avg_trav_dist_5pct, df_avg_trav_dist], axis = 0)


In [38]:
# 5 pct random global seed
avg_trav_time_5pct_rGs = pd.DataFrame()
avg_trav_dist_5pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # Create Data frame and insert first value from avg_trav_time_5pct
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time_5pct["avg_trav_time"].iloc[0], "sample_size": "5-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1])
        
        # Create Data frame and insert first value from avg_trav_dist_5pct
        df1 = pd.DataFrame({"avg_trav_dist":avg_trav_dist_5pct["avg_trav_dist"].iloc[0], "sample_size": "5-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df1])

    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_3254_3765/ITERS/it.0/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_3254_3765.0.trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "5-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = np.mean(temp["traveled_distance"])
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df_avg_trav_dist], axis = 0)

    else:
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/ITERS/it.0/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765.0.trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "5-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = np.mean(temp["traveled_distance"])
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df_avg_trav_dist], axis = 0)


In [39]:
avg_trav_time_5pct_all = pd.concat([avg_trav_time_5pct, avg_trav_time_5pct_rGs], axis = 0 , ignore_index= True)
avg_trav_time_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_5pct_samples_it0.csv', index = False) 
avg_trav_dist_5pct_all = pd.concat([avg_trav_dist_5pct, avg_trav_dist_5pct_rGs ], axis = 0 , ignore_index= True)
avg_trav_dist_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_5pct_samples_it0.csv', index = False) 

##### 10 pct

In [40]:
flowCapF = ["0.1"]
storCapF =  ["0.1", "0.17783"]
stuckTimes = ["30.0", "300.0"]

avg_trav_time_10pct = pd.DataFrame()
avg_trav_dist_10pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,11,1):
                # declare path based on case 
                if((fCf == "0.1") & (sCf == "0.1") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" +str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) +"-fCf_sCF_0.1_gS_4711_3765.0.trips.csv.gz"       
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_3765.0.trips.csv.gz"
                elif((fCf == "0.1") & (sCf == "0.1") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.0.trips.csv.gz"
                elif((fCf == "0.1") & (sCf ==  "0.17783") & (sT == "300.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/ITERS/it.0/lausitz-10-pct-" + str(sampleNr) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.0.trips.csv.gz"  
                else: 
                    print("case not found")
                    break
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                if(sCf == "0.17783"):
                    alpha = 0.75
                else: 
                    alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "10-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_10pct = pd.concat([avg_trav_time_10pct, df_avg_trav_time], axis = 0)
                # calculate average traveled distance
                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "10-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_10pct = pd.concat([avg_trav_dist_10pct, df_avg_trav_dist], axis = 0)


/tmp/ipykernel_5897/1327652803.py:24: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_5897/1327652803.py:24: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_5897/1327652803.py:24: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_5897/1327652803.py:24: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_5897/1327652803.py:24: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/

In [41]:
avg_trav_time_10pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_10pct_samples_it0.csv', index = False) 
avg_trav_dist_10pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_10pct_samples_it0.csv', index = False) 

##### 25 pct

In [42]:
flowCapF = ["0.25"]
storCapF =  ["0.25", "0.35355"]
stuckTimes = ["30.0", "120.0"]

avg_trav_time_25pct = pd.DataFrame()
avg_trav_dist_25pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.25") & (sCf == "0.25") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.0.trips.csv.gz"       
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.0.trips.csv.gz"
                elif((fCf == "0.25") & (sCf == "0.25") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/ITERS/it.0/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.0.trips.csv.gz"
                elif((fCf == "0.25") & (sCf ==  "0.35355") & (sT == "120.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/ITERS/it.0/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.0.trips.csv.gz"  
                else: 
                    print("case not found")
                    break
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                if(sCf == "0.35355"):
                    alpha = 0.75
                else: 
                    alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "25-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_25pct = pd.concat([avg_trav_time_25pct, df_avg_trav_time], axis = 0)
                # calculate average traveled distance
                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "25-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_25pct = pd.concat([avg_trav_dist_25pct, df_avg_trav_dist], axis = 0)



In [44]:
avg_trav_time_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_25pct_samples_it0.csv', index = False) 
avg_trav_dist_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_25pct_samples_it0.csv', index = False) 

##### 50 pct

In [45]:
flowCapF = ["0.5"]
storCapF =  ["0.5", "0.5946"]
stuckTimes = ["30.0", "60.0"]

avg_trav_time_50pct = pd.DataFrame()
avg_trav_dist_50pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                if((fCf == "0.5") & (sCf == "0.5") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.0.trips.csv.gz"       
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "30.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.0.trips.csv.gz"
                elif((fCf == "0.5") & (sCf == "0.5") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.0.trips.csv.gz"
                elif((fCf == "0.5") & (sCf ==  "0.5946") & (sT == "60.0")):
                    path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/ITERS/it.0/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.0.trips.csv.gz"  
                else: 
                    print("case not found")
                    break
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                if(sCf == "0.5946"):
                    alpha = 0.75
                else: 
                    alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "50-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_50pct = pd.concat([avg_trav_time_50pct, df_avg_trav_time], axis = 0)
                # calculate average traveled distance
                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "50-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_50pct = pd.concat([avg_trav_dist_50pct, df_avg_trav_dist], axis = 0)



In [46]:
avg_trav_time_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_50pct_samples_it0.csv', index = False) 
avg_trav_dist_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_50pct_samples_it0.csv', index = False) 

##### 100 pct

In [48]:
flowCapF = ["1.0"]
storCapF =  ["1.0"]
stuckTimes = ["30.0"]

avg_trav_time_100pct = pd.DataFrame()
avg_trav_dist_100pct = pd.DataFrame()
counter = 0
for fCf in flowCapF:
    for sCf in storCapF:
        for sT in stuckTimes:
            for sampleNr in range(1,2,1):
                # declare path based on case 
                path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.0.trips.csv.gz"
                temp = pd.read_csv(path, compression= "gzip", sep=";")
                temp["trav_time"] = pd.to_timedelta(temp.trav_time)
                avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
                alpha = 1.0

                df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "100-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_time_100pct = pd.concat([avg_trav_time_100pct, df_avg_trav_time], axis = 0)
                # calculate average traveled distance
                avg_trav_dist = np.mean(temp["traveled_distance"])
                df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "100-pct", "sample_nr": sampleNr, "alpha" : alpha, "stuck_time": float(sT), "global_seed": "default"}, index=[sampleNr])
                avg_trav_dist_100pct = pd.concat([avg_trav_dist_100pct, df_avg_trav_dist], axis = 0)

##### Concat and write output files

In [50]:
# concat all
avg_trav_time_1_100pct = pd.concat([avg_trav_time_1pct_all, avg_trav_time_5pct_all, avg_trav_time_10pct, avg_trav_time_25pct, avg_trav_time_50pct, avg_trav_time_100pct ], axis = 0)
# write all to csv
avg_trav_time_1_100pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_1_100pct_samples_it0.csv", index = False)

# concat all
avg_trav_dist_1_100pct = pd.concat([avg_trav_dist_1pct_all, avg_trav_dist_5pct_all, avg_trav_dist_10pct, avg_trav_dist_25pct, avg_trav_dist_50pct, avg_trav_dist_100pct ], axis = 0)
# write all to csv
avg_trav_dist_1_100pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_1_100pct_samples_it0.csv", index = False)



#### Travel Time distribution

In [6]:
def sixMinuteCategoriesUpToTwoHours(df):
    categories = []
    for t in df["trav_time2"]:


        if (0.0 <= t and t < 0.10):
            categories.append("0:00_0:06")
        elif (0.10 <= t and t < 0.20):
            categories.append("0:06_0:12") 
        elif (0.20 <= t and t < 0.30):
            categories.append("0:12_0:18")
        elif (0.30 <= t and t < 0.40):
            categories.append("0:18_0:24")
        elif (0.40 <= t and t < 0.50):
            categories.append("0:24_0:30")
        elif (0.50 <= t and t < 0.60):
            categories.append("0:30_0:36")  
        elif (0.60 <= t and t < 0.70):
            categories.append("0:36_0:42")
        elif (0.70 <= t and t < 0.80):
            categories.append("0:42_0:48")
        elif (0.80 <= t and t < 0.90):
            categories.append("0:48_0:54")
        elif (0.90 <= t and t < 1.00):
            categories.append("0:54_1:00")


        elif (1.0 <= t and t < 1.10):
            categories.append("1:00_1:06")
        elif (1.10 <= t and t < 1.20):
            categories.append("1:06_1:12") 
        elif (1.20 <= t and t < 1.30):
            categories.append("1:12_1:18")
        elif (1.30 <= t and t < 1.40):
            categories.append("1:18_1:24")
        elif (1.40 <= t and t < 1.50):
            categories.append("1:24_1:30")
        elif (1.50 <= t and t < 1.60):
            categories.append("1:30_1:36")  
        elif (1.60 <= t and t < 1.70):
            categories.append("1:36_1:42")
        elif (1.70 <= t and t < 1.80):
            categories.append("1:42_1:48")
        elif (1.80 <= t and t < 1.90):
            categories.append("1:48_1:54")
        elif (1.90 <= t and t < 2.00):
            categories.append("1:54_2:00")

        else: categories.append(">2h")

    temp = pd.DataFrame({ "categories": categories})
    temp = temp.sort_values(by=['categories'])

    freq_6_min_cat = []
    for cat in temp.categories.unique():
        f = temp[(temp["categories"]==cat)].shape[0]
        freq_6_min_cat.append(f)
    
    trav_times_6min_cat = pd.DataFrame({"category":temp.categories.unique(), "freq": freq_6_min_cat })
    return trav_times_6min_cat 

##### 5 pct

In [7]:
# 1pct

trav_time_1pct_categories = []
for sampleNr in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.output_trips.csv.gz
    path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_2306_3765/ITERS/it.0/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_2306_3765.0.trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
    trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
    trav_time_cat.rename(columns={"freq": "freq_" + str(sampleNr)}, inplace = True )
    trav_time_1pct_categories.append(trav_time_cat)
trav_time_1pct_cat_all = pd.merge(trav_time_1pct_categories[0], trav_time_1pct_categories[1], on = ['category'], how='left')
for sampleNr in range(2,10,1):
    trav_time_1pct_cat_all = pd.merge(trav_time_1pct_cat_all, trav_time_1pct_categories[sampleNr], on = ['category'], how='left')

trav_time_1pct_cat_all['mean'] = trav_time_1pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_1pct_cat_all.insert(12, 'sample_size', "1-pct")
trav_time_1pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_1pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_1pct_cat_all.insert(15,'global_seed', "default") 



##### 5 pct

In [8]:
trav_time_5pct_categories = []
for sampleNr in range(1,11,1):
    if (sampleNr == 6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.0.trips.csv.gz"
        temp = pd.read_csv(path, compression = "gzip", sep = ";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
        trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
        trav_time_cat.rename(columns={"freq": "freq_" + str(sampleNr)}, inplace = True )
        trav_time_5pct_categories.append(trav_time_cat)
    else:     
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" +str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765/ITERS/it.0/lausitz-5.0-pct-" + str(sampleNr) + "-fCf_sCF_0.05_gS_4711_3765.0.trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
        trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
        trav_time_cat.rename(columns={"freq": "freq_" + str(sampleNr)}, inplace = True )
        trav_time_5pct_categories.append(trav_time_cat)
# left join and calculate mean
trav_time_5pct_cat_all = pd.merge(trav_time_5pct_categories[0], trav_time_5pct_categories[1], on = ['category'], how='left')
for sampleNr in range(2,10,1):
    trav_time_5pct_cat_all = pd.merge(trav_time_5pct_cat_all, trav_time_5pct_categories[sampleNr], on = ['category'], how='left')

trav_time_5pct_cat_all['mean'] = trav_time_5pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_5pct_cat_all.insert(12, 'sample_size', "5-pct")
trav_time_5pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_5pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_5pct_cat_all.insert(15,'global_seed', "default") 

##### 10 pct

In [9]:
trav_time_10pct_categories = []
for sampleNr in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" +str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765/ITERS/it.0/lausitz-10.0-pct-" + str(sampleNr) + "-fCf_sCF_0.1_gS_4711_3765.0.trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
    trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
    trav_time_cat.rename(columns={"freq": "freq_" + str(sampleNr)}, inplace = True )
    trav_time_10pct_categories.append(trav_time_cat)
# left join and calculate mean
trav_time_10pct_cat_all = pd.merge(trav_time_10pct_categories[0], trav_time_10pct_categories[1], on = ['category'], how='left')
for sampleNr in range(2,10,1):
    trav_time_10pct_cat_all = pd.merge(trav_time_10pct_cat_all, trav_time_10pct_categories[sampleNr], on = ['category'], how='left')

trav_time_10pct_cat_all['mean'] = trav_time_10pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_10pct_cat_all.insert(12, 'sample_size', "10-pct")
trav_time_10pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_10pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_10pct_cat_all.insert(15,'global_seed', "default") 

/tmp/ipykernel_6256/3501379692.py:4: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_6256/3501379692.py:4: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")
/tmp/ipykernel_6256/3501379692.py:4: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


##### 25 pct

In [10]:
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/ITERS/it.0/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.0.trips.csv.gz"
t_25pct = pd.read_csv(path, compression= "gzip", sep=";")
t_25pct["trav_time"] = pd.to_timedelta(t_25pct.trav_time)
t_25pct["trav_time2"] = t_25pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_25pct_categories = sixMinuteCategoriesUpToTwoHours(t_25pct)
trav_time_25pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_25pct_categories.insert(2, 'sample_size', "25-pct")
trav_time_25pct_categories.insert(3, 'alpha', 1.0)
trav_time_25pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_25pct_categories.insert(5,'global_seed', "default") 

##### 50 pct

In [11]:
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/ITERS/it.0/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.0.trips.csv.gz"
t_50pct = pd.read_csv(path, compression= "gzip", sep=";")
t_50pct["trav_time"] = pd.to_timedelta(t_50pct.trav_time)
t_50pct["trav_time2"] = t_50pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_50pct_categories = sixMinuteCategoriesUpToTwoHours(t_50pct)
trav_time_50pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_50pct_categories.insert(2, 'sample_size', "50-pct")
trav_time_50pct_categories.insert(3, 'alpha', 1.0)
trav_time_50pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_50pct_categories.insert(5,'global_seed', "default") 

##### 100 pct

In [12]:
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/ITERS/it.0/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.0.trips.csv.gz"
t_100pct = pd.read_csv(path, compression= "gzip", sep=";")
t_100pct["trav_time"] = pd.to_timedelta(t_100pct.trav_time)
t_100pct["trav_time2"] = t_100pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_100pct_categories = sixMinuteCategoriesUpToTwoHours(t_100pct)
trav_time_100pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_100pct_categories.insert(2, 'sample_size', "100-pct")
trav_time_100pct_categories.insert(3, 'alpha', 1.0)
trav_time_100pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_100pct_categories.insert(5,'global_seed', "default") 

##### Concat and write output file

In [13]:
trav_time_categories_all = pd.concat([trav_time_1pct_cat_all, trav_time_5pct_cat_all, trav_time_10pct_cat_all,  trav_time_25pct_categories, trav_time_50pct_categories, trav_time_100pct_categories], axis = 0)
trav_time_categories_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/trav_time_categories_1_100_it0.csv', index = False) 